In [1]:
import random
import numpy as np
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import cv2
import sys
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
def generate_file_list(path,p):
    print("[INFO] loading images...")
    data = []
    labels = []
    train_y = []
    # grab the image paths and randomly shuffle them
    
    labels = os.listdir(path)
    #labels.remove('.DS_Store')
    imagePaths = []
    print(labels)
    for i, label in enumerate(labels):
        path_file=os.path.join(path, label)
        #path_file = os.path.join(path,'label')
        path_files = os.path.join(path_file,p)
        imagePath = os.listdir(path_files)
        #print(path_files)
        #imagePath.remove('.DS_Store')
        for img in imagePath:
            if img == '.DS_Store':
                print('wrong')
            else:
                image = os.path.join(path_files,img)
                imagePaths.append(image)
            #print(path_files)
    #print(imagePaths)
    random.seed(42)
    random.shuffle(imagePaths)
    
    return imagePaths

In [3]:
part='train'
trainlist = generate_file_list(path='backup/',p=part) 


[INFO] loading images...
['ACTION', 'ALLEGATIONS', 'AFFAIRS', 'ABUSE', 'ALLOWED', 'ALLOW']


# write

In [4]:
fileObject = open('filelist.txt', 'w')
for ip in trainlist:
    fileObject.write(ip)
    fileObject.write('\n')
fileObject.close()

In [5]:
train_list = []
fd = open( "filelist.txt", "r" )

for line in fd.readlines():
    train_list.append(line.rstrip('\n'))
    #print(line)
#print(train_list)

In [6]:
def load_data_GRAY(path,p):
    print("[INFO] loading images...")
    data = []
    labels = []
    train_y = []
    # grab the image paths and randomly shuffle them
    
    labels = os.listdir(path)
    #labels.remove('.DS_Store')
    imagePaths = []
    print(labels)
    for i, label in enumerate(labels):
        path_file=os.path.join(path, label)
        #path_file = os.path.join(path,'label')
        path_files = os.path.join(path_file,p)
        imagePath = os.listdir(path_files)
        #print(path_files)
        #imagePath.remove('.DS_Store')
        for img in imagePath:
            if img == '.DS_Store':
                print('wrong')
            else:
                image = os.path.join(path_files,img)
                imagePaths.append(image)
            #print(path_files)
    #print(imagePaths)
    random.seed(42)
    random.shuffle(imagePaths)
    #print(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        image_path = sorted(list(paths.list_images(imagePath)))
        images=[]
        for img in image_path:
            
            # load the image, pre-process it, and store it in the data list
            image = cv2.imread(img)
            #image = cv2.resize(image, (norm_size, norm_size))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            #image = cv2.resize(image, (norm_size, norm_size))
            image = img_to_array(image)
            #print(image)
            #image = np.array(image, dtype="float") / 255.0
            #print(image.shape)
            image = np.reshape(image,(120,120))
            images.append(image)
        
        images=np.array(images,dtype="float")
        if(len(images)==0):
            print(imagePath)
            break
        #images=np.reshape(images,(120,120,29))
        #print(images.shape)
        #print(images.shape)
        data.append(images)

        # extract the class label from the image path and update the
        # labels list
        #label = int(imagePath.split(os.path.sep)[-2])
        basename = os.path.basename(imagePath)
        label = basename.split('_')
        train_y.append(label[0])
        #print(len(labels))
    #data = np.array(data, dtype="float") / 255.0
    data = np.array(data, dtype="float")
    #print(data.shape)

    # scale the raw pixel intensities to the range [0, 1]
    train_y = np.array(train_y)
    print(train_y.shape)
    # 整数编码
    label_encoder = LabelEncoder()
    integer_result = label_encoder.fit_transform(train_y)
    # One-Hot编码
    one_hot_encoder = OneHotEncoder()
# One-Hot编码将分类值映射到整数值再表示成二进制向量
    integer_result1 = integer_result.reshape(len(integer_result), 1)
    one_hot_result = one_hot_encoder.fit_transform(integer_result1)

    #labels = np.array(labels)

    # convert the labels from integers to vectors
    #labels = to_categorical(labels, num_classes=CLASS_NUM)                         
    return data,one_hot_result

In [7]:
part='val'
valX,valY = load_data_GRAY(path='backup/',p=part) 

part='test'
testX,testY = load_data_GRAY(path='backup/',p=part) 

[INFO] loading images...
['ACTION', 'ALLEGATIONS', 'AFFAIRS', 'ABUSE', 'ALLOWED', 'ALLOW']
(300,)
[INFO] loading images...
['ACTION', 'ALLEGATIONS', 'AFFAIRS', 'ABUSE', 'ALLOWED', 'ALLOW']
(300,)


In [8]:
val_Y=valY.toarray()
test_Y = testY.toarray()

In [9]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import LSTM
from keras import backend as K
from keras.layers.wrappers import TimeDistributed
#from keras.layers.extra import TimeDistributedConvolution2D, TimeDistributedMaxPooling2D, TimeDistributedFlatten

In [10]:
class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
        		# CONV => RELU => POOL
		model.add(Conv2D(96, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
		model.add(BatchNormalization())
		model.add(MaxPooling2D(pool_size=(3, 3)))
		#model.add(Dropout(0.25))
        		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
		model.add(BatchNormalization())
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Conv2D(512, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		#model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.25))
        		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(512, (3, 3), padding="same"))
		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
		model.add(BatchNormalization())
		model.add(MaxPooling2D(pool_size=(3, 3)))
		#model.add(Conv2D(128, (3, 3), padding="same"))
		#model.add(Activation("relu"))
		#model.add(BatchNormalization(axis=chanDim))
		#model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.25))
        		# first (and only) set of FC => RELU layers
# 		model.add(TimeDistributed(Flatten())
# 		model.add(LSTM(512, return_sequences=True))
# 		model.add(BatchNormalization())
# 		model.add(LSTM(12, return_sequences=True))
# 		model.add(BatchNormalization())
		model.add(Flatten())
		model.add(Dense(4096))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		#model.add(Dropout(0.5))
		model.add(Dense(4096))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
        
		# return the constructed network architecture
		return model

In [11]:
EPOCHS = 3
INIT_LR = 0.1
BS = 16
IMAGE_DIMS = (120,120,29)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [12]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=6)
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [14]:
valX = np.reshape(valX,(-1,120,120,29))
testX = np.reshape(testX,(-1,120,120,29))

In [15]:
Y = os.listdir("backup/")

In [16]:
label_encoder = LabelEncoder()
integer_result = label_encoder.fit(Y)
integer_result = label_encoder.transform(Y)
# One-Hot编码
one_hot_encoder = OneHotEncoder()
# One-Hot编码将分类值映射到整数值再表示成二进制向量
integer_result1 = integer_result.reshape(len(integer_result), 1)
#integer_result1 = integer_result1.flatten()
Y = one_hot_encoder.fit(integer_result1)

In [17]:
Y = one_hot_encoder.transform(integer_result1)

In [18]:
Y.toarray()

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [19]:
def generate_arrays_from_file(path,batch_size):  
    while 1:  
        f = path
        cnt = 0  
        X =[]  
        Y =[]  
        for line in f:  
            #print(line)
            # create Numpy arrays of input data  
            # and labels, from each line in the file  
            image_path = sorted(list(paths.list_images(line)))
            images=[]
            for img in image_path:

                # load the image, pre-process it, and store it in the data list
                image = cv2.imread(img)
                #image = cv2.resize(image, (norm_size, norm_size))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                #image = cv2.resize(image, (norm_size, norm_size))
                image = img_to_array(image)
                #print(image)
                #image = np.array(image, dtype="float") / 255.0
                #print(image.shape)
                image = np.reshape(image,(120,120))
                images.append(image)
 
            images = np.array(images,dtype="float")
            if(len(images)==0):
                print(imagePath)
                break
            #images=np.reshape(images,(120,120,29))
            #print(images.shape)
            #print(images.shape)
            X.append(images)

            # extract the class label from the image path and update the
            # labels list
            #label = int(imagePath.split(os.path.sep)[-2])
            basename = os.path.basename(line)
            label = basename.split('_')
            Y.append(label[0]) 
            
            #X = np.array(X, dtype="float")
            #print(data.shape)

            # scale the raw pixel intensities to the range [0, 1]

            
            cnt += 1  
            if cnt==batch_size:  
                cnt = 0 
                #print(Y)
                #Y = np.array(Y)
                #print(Y.shape)
                # 整数编码
                integer_result = label_encoder.transform(Y)
                #print(integer_result)
                integer_result1 = integer_result.reshape(len(integer_result), 1)
                Y = one_hot_encoder.transform(integer_result1)
                Y = Y.toarray()
                #print(Y.shape)
                
                X = np.array(X, dtype="float")
                X = np.reshape(X,(-1,120,120,29))
                #print(X.shape)
                yield (X, Y)  
                X = []  
                Y = []  
    f.close() 

In [21]:
print("[INFO] training network...")
H = model.fit_generator(
	generate_arrays_from_file(train_list,batch_size=64),
	validation_data=(valX, val_Y),
	steps_per_epoch=5900 // 64,
	epochs=5, verbose=1)

[INFO] training network...
Epoch 1/5
12/92 [==>...........................] - ETA: 150s - loss: 0.1103 - acc: 0.9753

KeyboardInterrupt: 

In [22]:
model.evaluate(testX,test_Y)

300/300 [==============================] - 3s     


[0.9814994224905967, 0.6866666666666666]